In [1]:
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


# Load the dataset
df = pd.read_csv('/kaggle/input/mimic-mp/MP_IN_adm_train.csv')

# Load the pretrained model and tokenizer
model_name = 'shhossain/opus-mt-en-to-bn'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/969k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.05M [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/303M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(61760, 512, padding_idx=61759)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(61760, 512, padding_idx=61759)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [2]:
import re
from typing import Optional

def preprocess_text(text: str, remove_headers: bool = True) -> str:
    """
    Preprocess clinical notes while preserving important medical information.
    
    Args:
        text: The input clinical note text
        remove_headers: Whether to remove section headers like "CHIEF COMPLAINT:", "ALLERGIES:", etc.
    
    Returns:
        Preprocessed text string
    """
    # Replace multiple newlines/spaces with single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove anonymized placeholders (e.g., [**Name**], [**Hospital1**])
    text = re.sub(r'\[\*\*.*?\*\*\]', '', text)
    
    if remove_headers:
        # Remove common clinical note headers
        headers = [
            'CHIEF COMPLAINT:',
            'PRESENT ILLNESS:',
            'MEDICAL HISTORY:',
            'MEDICATION ON ADMISSION:',
            'ALLERGIES:',
            'PHYSICAL EXAM:',
            'FAMILY HISTORY:',
            'SOCIAL HISTORY:'
        ]
        for header in headers:
            text = text.replace(header, '')
    
    # Standardize some common medical abbreviations (expand if needed)
    text = text.replace(' yo ', ' year old ')
    text = text.replace(' y/o ', ' year old ')
    text = text.replace(' hr ', ' heart rate ')
    text = text.replace(' bp ', ' blood pressure ')
    text = text.replace(' temp ', ' temperature ')
    text = text.replace(' rx ', ' prescription ')
    text = text.replace(' hx ', ' history ')
    text = text.replace(' dx ', ' diagnosis ')
    text = text.replace(' sx ', ' symptoms ')
    
    # Keep special characters that might be meaningful in medical context
    # Like +/-, %, °, vital signs measurements
    text = re.sub(r'[^\w\s+/\-=%°\.]', ' ', text)
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    return text


In [3]:
# Apply the preprocessing function to the text column
text = preprocess_text(df['text'][0])

In [4]:
print(text)

AMS concern for toxic alcohol ingestion Mr. is a 39 year old man with h/o EtOH and polysubstance abuse seizure disorder who was found to be unresponsive while visiting his partner in the ICU earlier today. . The patient was visiting his partner in the ICU earlier today. He was awake and conversant in the morning with no acute complaints. He was noted to be sleeping on the floor but walked to the chair by himself when he was awakened. Later in the afternoon the patient was noted to still be asleep in the chair. He was unarousable with verbal stimuli or sternal rub so he was taken down to the ED. . In the ED the patient was initially altered but was otherwise hemodynamically stable. No urine incontinence or e/o toxidromes. Labs notable for EtOH 86 Osms 366 anion gap 16 lactate 3.8. Utox positive for barbs but Stox and Utox otherwise negative. Given high serum osmolar gap 60 toxicology was consulted for concern of toxic alcohol ingestion. Most likely isopropyl alcohol given osmolar gap wi

In [5]:
# Function to translate text
def translate(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length = 512).to(device)
    translated = model.generate(**inputs)
    translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return translated_text[0]

In [8]:
from tqdm import tqdm

# Function to split text into chunks by word count
def split_text_into_chunks(text, words_per_chunk=30):
    words = text.split()
    chunks = []
    
    for i in range(0, len(words), words_per_chunk):
        chunk = ' '.join(words[i:i + words_per_chunk])
        chunks.append(chunk)
    
    return chunks

# Function to translate text by chunks with preprocessing
def translate_text_by_chunks_with_preprocessing(text):
    # First preprocess the text
    # processed_text = preprocess_text(str(text))

    processed_text = text
    # Split processed text into chunks
    chunks = split_text_into_chunks(processed_text)
    
    # Translate each chunk
    translated_chunks = []
    for chunk in chunks:
        if chunk.strip():  # Only translate non-empty chunks
            try:
                translated = translate(chunk)
                translated_chunks.append(translated)
            except Exception as e:
                print(f"Error translating chunk: {e}")
                translated_chunks.append("")
    
    # Join the translated chunks
    return " ".join(translated_chunks)

In [9]:
# First, separate the data by class and sample 1000 from each
class_0_samples = df[df['hospital_expire_flag'] == 0].sample(n=1000, random_state=42)
class_1_samples = df[df['hospital_expire_flag'] == 1].sample(n=1000, random_state=42)

# Combine the samples into a new balanced dataframe
balanced_df = pd.concat([class_0_samples, class_1_samples]).reset_index(drop=True)

# Create a new dataframe with the columns we need
final_df = pd.DataFrame({
    'english_text': balanced_df['text'],  # Replace 'text' with your text column name
    'english_text_preprocessed': None,  # New column for preprocessed text
    'label': balanced_df['hospital_expire_flag']
})

# First, preprocess all texts
print("Preprocessing texts...")
for idx in tqdm(range(len(final_df)), desc="Preprocessing"):
    try:
        original_text = str(final_df.iloc[idx]['english_text'])
        preprocessed = preprocess_text(original_text)
        final_df.at[idx, 'english_text_preprocessed'] = preprocessed
    except Exception as e:
        print(f"Error preprocessing row {idx}: {e}")
        final_df.at[idx, 'english_text_preprocessed'] = ""

# Now translate the preprocessed texts
print("\nTranslating preprocessed texts...")
final_df['bengali_text'] = None
for idx in tqdm(range(len(final_df)), desc="Translating"):
    try:
        preprocessed_text = str(final_df.iloc[idx]['english_text_preprocessed'])
        translated = translate_text_by_chunks_with_preprocessing(preprocessed_text)
        final_df.at[idx, 'bengali_text'] = translated
    except Exception as e:
        print(f"Error processing row {idx}: {e}")
        final_df.at[idx, 'bengali_text'] = ""

# Save the results
output_path = 'balanced_translated_dataset_preprocessed.csv'
final_df.to_csv(output_path, index=False)
print(f"\nTranslations completed and saved to {output_path}")

Preprocessing texts...


Preprocessing: 100%|██████████| 2000/2000 [00:01<00:00, 1926.39it/s]



Translating preprocessed texts...


Translating: 100%|██████████| 2000/2000 [3:48:15<00:00,  6.85s/it]  



Translations completed and saved to balanced_translated_dataset_preprocessed.csv


In [14]:
final_df['bengali_text'][0]

'প্রধান অভিযোগ হল, দুর্দশাগ্রস্ত রোগ বর্তমান পুরুষ ইতিহাসের পুরাতন ধারাবাহিক প্রাক্\u200cনিসেন এমটিক্স কালের নির্বাচনী রাষ্ট্রীয় স্বাস্থ্যের হার কমে আসার কারণে গতকাল সন্ধ্যায় ক্ষয়ক্ষতির দুর্বলতা ভোগ করতে শুরু করে। গ্লুকোজ লেভেল ঠাণ্ডা ইনসুলিন যা কিছুই খেতে পারে না, যা কিছু ঘুমিয়ে পড়ে, তাই এমএসকে বলা দুর্বল ড্রাইভ হাসপাতাল ড্রার নিয়ে আসে, যেখানে রক্তচাপের এমএস রেকর্ড করা হয়েছে। আসা sbp প্রদত্ত lhuly minimal প্রতিক্রিয়া কেন্দ্রীয় লাইনে lvofed respect lvefed শুরু করে বর্তমান জরুরী hr bp rr l nc প্রদত্ত চাপ dose hudiocortisne josyn vanco Asperine সম্ভাব্য স্টেট ডিসেম্বর ভিভি রোস ইতিবাচক স্বল্পস্বাস্থ্য কাশির হুমকির মুখে নাসল স্প্রে পেরিয়ে নেসাল স্প্রে ক্ষয়িত রোগী জ্বরকে অস্বীকার করে ওজন বমিতে বমি ব্যথার ব্যথা ব্যথা কন্সটিপশন মেলেনা হেমাটোচিজিয়া বুকে ব্যথার যন্ত্রণার যন্ত্রপাতি অসহিষ্ণু ত্বক চিকিত্সা ইতিহাসকে পরিবর্তন করে। গত বছর অপ্রতিক্রিয়তার পর থেকেই ইমুরান রিমিকেড পরীক্ষা ইমুরান এমটিক্সের অতীত পরীক্ষায় পরিণত হয়েছে, কারণ remicade sis se readnison mg সর্বশেষ dose pcp prophy

In [10]:
# Print some statistics
print("\nDataset Statistics:")
print(f"Total samples: {len(final_df)}")
print("\nClass distribution:")
print(final_df['label'].value_counts())

# Print sample translations
print("\nSample translations (first 2 rows):")
for idx in range(2):
    print(f"\nExample {idx + 1}:")
    print(f"Original English: {final_df.iloc[idx]['english_text'][:200]}...")
    print(f"Preprocessed English: {final_df.iloc[idx]['english_text_preprocessed'][:200]}...")
    print(f"Bengali: {final_df.iloc[idx]['bengali_text'][:200]}...")


Dataset Statistics:
Total samples: 2000

Class distribution:
label
0    1000
1    1000
Name: count, dtype: int64

Sample translations (first 2 rows):

Example 1:
Original English: CHIEF COMPLAINT: weak/fatigue

PRESENT ILLNESS: This is a 72 year-old male with a history of UC on chronic prednisone and MTX, scheduled for elective colectomy this am. He was in his usual state of he...
Preprocessed English: chief complaint weakfatigue present illness yearold male history uc chronic prednisone mtx scheduled elective colectomy usual state health c yesterday started experiencing worstening fatigue weakness ...
Bengali: ...

Example 2:
Original English: CHIEF COMPLAINT: Fever and hypotension.

PRESENT ILLNESS: The patient was seen in the Emergency Department on [**2147-8-4**] with the onset of fever and hypotension.  The patient is a 44-year-old whit...
Preprocessed English: chief complaint fever hypotension present illness patient seen emergency department onset fever hypotension patient year

In [8]:
final_df['english_text'][0]

'CHIEF COMPLAINT: weak/fatigue\n\nPRESENT ILLNESS: This is a 72 year-old male with a history of UC on chronic prednisone and MTX, scheduled for elective colectomy this am. He was in his usual state of health until c yesterday when he started experiencing worstening fatigue and weakness. He had no po intake through out most of the day and in the evening had a very low glucose level (?) and some chills, he had taken his insulin but not eaten anything. He did not do anything about this but lay down to sleep. This am he felt too weak to drive to hospital by himself and called EMS to bring him for his surgery to be done by Dr. [**Last Name (STitle) **], [**First Name3 (LF) **]. Upon arrival EMS recorded a blood pressure of 60/40, FS 118, got fluid and upon arrival in ED SBP of 80-90, he was given 5 L fluid with some minimal response response, central line was placed, Levophed was started. Tmax of around 100, current vitals HR 70 BP 96/53 RR 22 O2 95% 4L NC He was given stress dose Hydrocort

In [9]:
final_df['bengali_text'][0]

'CHIEF COMPLACINT: দুর্বল/ফরগমেন্ট প্রেসেন্ট: এটা ৭২ বছর বয়সী একজন পুরুষ যার দীর্ঘস্থায়ী পূর্বাভাস ও এমটিএক্সের ইতিহাস ছিল। গতকাল পর্যন্ত তার স্বাস্থ্যের স্বাভাবিক অবস্থা ছিল, যখন তিনি অত্যন্ত দুর্বলতা ও দুর্বলতা ভোগ করতে শুরু করেছিলেন। দিনের বেশিরভাগ সময় ও দিনের মধ্যে তার কোন অস্থিরতা ছিল না। সন্ধ্যায় গ্লুকোজের মাত্রা খুব কম ছিল (??) এবং কিছু ঠাণ্ডা ছিল, তিনি তার ইনসুলিন নিয়েছিলেন কিন্তু কিছু খেতেননি। ঘুমানোর জন্য তিনি নিজেকে হাসপাতালে নিয়ে যাওয়াকে খুব দুর্বল বলে মনে করেন এবং ডক্টর ড. [*শেষ নাম] তার সার্জারির জন্য তাকে আনার জন্য ইএমএস নামে ডাকেন। (STitle) ** [**] [** প্রথম নাম ৩ (এলএফ)**)। ইএমএস এসে ৬০/৪০, এফএস ১১৮ এর রক্তচাপ রেকর্ড করে, এবং ৮০-৯০ সালের এডিবিপিতে পৌঁছায়, তাকে দেওয়া হয়। ৫ L তরল কিছু সামান্য প্রতিক্রিয়ার সাথে, কেন্দ্রীয় লাইন স্থাপন করা হয়েছিল, লেভোফেড শুরু হয়েছিল ১০০ এর প্রায়, বর্তমান গুরুত্বপূর্ণ এইচআর ৭০ বিপি ৯৬/৫৩ R ২২ ও ৯৫% ৪এল। এনসি তাকে ডোজ হাইড্রোকোর্টিসন, জোসিন, ভ্যানকো, অ্যাস্পিনিন (ভি২-ভি৪).আরওএস: শ্বাসের স্বল্পতার জন্য ইতিবাচক, ৩ উইক এবং নাসাল 

In [11]:
final_df['english_text'][1]

'CHIEF COMPLAINT: Fever and hypotension.\n\nPRESENT ILLNESS: The patient was seen in the Emergency Department on [**2147-8-4**] with the onset of fever and hypotension.  The patient is a 44-year-old white gentleman with a past medical history of end-stage renal disease secondary to ureteral reflux nephropathy.  He underwent a living-related renal transplant and has a history of mesenteric ischemia requiring a [**Doctor Last Name 4726**]-Tex superior mesenteric artery aorta bypass graft in [**2145-12-25**].  The patient now presents after a prior admission for occlusion, status post t-PA, of the superior mesenteric artery with a 24-hour to 36-hour history of increasing weakness, malaise, and fever.  Temperature was 102.8.  The patient admits to chills.  He denies chest pain, shortness of breath, or cough.  There was no bright red blood per rectum. He denies any associated symptoms or abdominal discomfort or pain.  He is now admitted for further evaluation and treatment.\n\nMEDICAL HISTO

In [10]:
final_df['bengali_text'][1]

'CHIEF COMPLAINT: Fever and hypension. Present ILLESTE: [*2147-8-4*] রোগীর জ্বর ও হাইপোটেনশনের উপর জ্বর ও হাইপোনেশনের উপর জরুরি বিভাগে রোগীকে দেখা হয়েছিল। রোগী ৪৪ বছর বয়সী সাদা। ভদ্রমহোদয়গণের প্রাক্তন মেডিকেল ইতিহাসের সার্ভেন্দ্রিক সংক্রমণের মধ্য দিয়ে নিয়াফ্রোপ্যাথির পুনর্বাসন। তিনি একটি জীবন্ত পুনর্গঠন গ্রহণ করেন এবং মেসেন্ট্রেনিক আইসচিমিয়ার একটি ইতিহাস প্রয়োজন। [**Doctor Last নেম ৪৭২৬**] [*২১৪-১২-২৫*]-টেক্স উচ্চতর মস্তিষ্কীয় ধূমপানের শ্বাসরোধী বাইপাস বাইপাস (**২১৪-১২-২৫*)। রোগী এখন অগ্ল্যুশনের জন্য প্রাক্তন ভর্তি, স্ট্যাটাস পোস্ট টি-পিএ, উচ্চতর মেসেন্টারিক ধূমপানের ক্রমবর্ধমান দুর্বলতা, মালিয়াস এবং জ্বরের চব্বিশ ঘন্টার মধ্যে ৩৬ ঘন্টার ইতিহাস। তাপমাত্রা ছিল ১০২.৮। রোগী স্বীকার করেন শীতল হওয়া। তিনি বুকের ব্যথা, শ্বাসের স্বল্পতা, বা কাশির কথা অস্বীকার করেন। তিনি কোন সংস্পর্শিত লক্ষণ বা অস্বস্তি অথবা ব্যথাকে অস্বীকার করেন। তিনি এখন আরো মূল্যায়ন ও চিকিৎসার জন্য ভর্তি হন। এমডিক্যাল ইতিহাস: ১. ইতিহাস ২। গ্যাসট্রোসোফেজিয়াল রোগের ইতিহাস. ৪। ধূমপানের ২৫ বছরের ইতিহাস। ৫. স্কোয়ামস ক

In [39]:
# First, separate the data by class and sample 1000 from each
class_0_samples = df[df['hospital_expire_flag'] == 0].sample(n=1000, random_state=42)
class_1_samples = df[df['hospital_expire_flag'] == 1].sample(n=1000, random_state=42)

# Combine the samples into a new balanced dataframe
balanced_df = pd.concat([class_0_samples, class_1_samples]).reset_index(drop=True)

# Create a new column for translations
balanced_df['translated_text'] = None

# Function to split text into segments with overlap
def split_text_with_overlap(text, window_size=200, overlap=50):
    if len(text) <= window_size:
        return [text]
    
    segments = []
    start = 0
    
    while start < len(text):
        # If we're at the last segment, just take the rest of the text
        if start + window_size >= len(text):
            segments.append(text[start:])
            break
            
        # Find the last space within the window to avoid cutting words
        end = start + window_size
        if end < len(text):
            # Look for the last space in the window
            last_space = text.rfind(' ', start, end)
            if last_space != -1:
                end = last_space
        
        segments.append(text[start:end])
        # Move the start position, accounting for overlap
        start = end - overlap
    
    return segments

# Function to translate long text using sliding window
def translate_long_text(text):
    # First preprocess the text
    processed_text = preprocess_text(text)
    
    # If the text is empty after preprocessing, return empty string
    if not processed_text.strip():
        return ""
    
    # Split into segments
    segments = split_text_with_overlap(processed_text)
    
    # Translate each segment
    translated_segments = []
    for segment in segments:
        try:
            translated_segment = translate(segment)
            translated_segments.append(translated_segment)
        except Exception as e:
            print(f"Error translating segment: {e}")
            translated_segments.append("")
            
    # Join the translated segments
    return " ".join(translated_segments)

In [40]:
df.head()

,id,text,hospital_expire_flag,translated_text
0,107384,"CHIEF COMPLAINT: AMS, concern for toxic alcoho...",0,"প্রধান অভিযোগ হল, বিষাক্ত মদ ব্যথার জন্য উদ্বে..."
1,101061,CHIEF COMPLAINT: abdominal pain\n\nPRESENT ILL...,0,প্রধান অভিযোগ এবডোমিনাল ব্যথা বর্তমান অসুস্থতা...
2,127180,CHIEF COMPLAINT: Bilateral Sub Dural Hematoma\...,0,প্রধান অভিযোগ: দ্বিপাক্ষিক উপ-ধারাল হেমাটোমা অ...
3,168339,CHIEF COMPLAINT: Intracranial bleed\n\nPRESENT...,0,রক্তক্ষরণের প্রধান অভিযোগ বর্তমান বৎসরের রোগে ...
4,154044,CHIEF COMPLAINT: ischemic left foot\n\nPRESENT...,0,"মুখ্য অভিযোগের মূল কারণ হল, একাডেমিকের জীবাণুগ..."


In [41]:
# Process all texts with tqdm progress bar
from tqdm import tqdm

# Create a new dataframe with only the columns we need
final_df = pd.DataFrame({
    'english_text': balanced_df['text'],  # Replace 'text' with your text column name
    'label': balanced_df['hospital_expire_flag']
})

# Add translations with progress bar
print("Translating texts...")
final_df['bengali_text'] = None
for idx in tqdm(range(len(final_df)), desc="Translating"):
    try:
        original_text = str(final_df.iloc[idx]['english_text'])
        translated = translate_long_text(original_text)
        final_df.at[idx, 'bengali_text'] = translated
    except Exception as e:
        print(f"Error processing row {idx}: {e}")
        final_df.at[idx, 'bengali_text'] = ""

# Save the results
output_path = 'balanced_translated_dataset.csv'
final_df.to_csv(output_path, index=False)
print(f"\nTranslations completed and saved to {output_path}")

Translating texts...


Translating:   0%|          | 2/2000 [00:16<4:39:14,  8.39s/it]


KeyboardInterrupt: 

In [42]:
final_df['bengali_text'][0]

'প্রধান অভিযোগ হল, দুর্দশাগ্রস্ত অসুস্থতা বর্তমান বছর বয়সী পুরুষ ইতিহাসের পুরাতন ধারাবাহিক প্রাক্\u200cনিসেন এমটিক্স কালকালের নির্বাচনী রাষ্ট্রীয় স্বাস্থ্যগত দুর্বলতা মোকাবেলা করতে শুরু করে। y sucking district po secting districts po secting po reseed see not glucks level should you insulture se duck you se spective secure spectial এমএস নামে দুর্বল ড্রাইভ হাসপাতালে রক্তচাপ ফ্লাওয়ার ফ্লাওয়ার পাওয়ার পর রক্তচাপ ফ্লাওয়ার সেলর পাওয়া গেছে সেন্ট্রাল লাইনের ন্যূনতম প্রতিক্রিয়া কেন্দ্রীয় রেলফ্রেশনে luid ন্যূনতম প্রতিক্রিয়া কেন্দ্রীয় প্রতিক্রিয়া কেন্দ্রীয় লাইনে lvefed বর্তমান creates hr bpr l nc প্রদত্ত চাপ dose hordicortisne zossyn vnco marspein সম্ভাব্য st vv ros জোসিন ভ্যানকো অ্যাসপিরিন সম্ভাব্য স্ট ডিস ডিগ্রি ভিভি রোস ইতিবাচক স্বল্পস্বল্পতা কাশির হুঁশ কাশির কারণ। আয় পোস্ট নাসাল ঢাকনা রোগী জ্বরকে অস্বীকার করে। ইয়া বুকে ব্যথা অটোপনেয়া পিন্ড নিম্ন সীমা ওডেমা ময়ূরি ফ্রিকোয়েন্সি জরুরি ডিসুরিয়া হালকা মাথার ঢাকা দৃষ্টিশক্তি মাথার মাথার মাথায় পরিবর্তন করে চিকিৎসা সংক্রান্ত ইতিহাস

In [ ]:
# Print some statistics
print("\nDataset Statistics:")
print(f"Total samples: {len(final_df)}")
print("\nClass distribution:")
print(final_df['label'].value_counts())

In [25]:
df['translated_text'][0]

'প্রধান অভিযোগ হল, বিষাক্ত মদ ব্যথার জন্য উদ্বেগজনক রোগব্যাধি বিদ্যমান বয়স্ক পুরুষ হোস পলিউসস্ট্যান্সের নির্যাতনের অসুবিধা আজ আগে রোগী আইকুর সাথে দেখা করতে যাওয়া সহযোগিতাকারী আইকুর সাথে দেখা করতে যাওয়া রোগীর কাছে অস্বীকার করা হয়েছে। আজ সকালে ঘুমন্ত তলায় অত্যন্ত তীব্র অভিযোগ উল্লেখ করা হয়েছে। রিংয়ের তীব্র অভিযোগে বলা হয়েছে যে, ঘুমন্ত মেঝের চেয়ার পরে ঘুমানো চেয়ারটি জেগে উঠেছে। দুপুরের রোগীর মতে ঘুমানো চেয়ারটি এখনও অসাধারণ মৌখিক মৌখিক স্ট্রিমাল রুব নিয়ে যাওয়া শুরুর দিকে পরিবর্তিত হয়েছে। অন্যথায় হেমোডামিটিক স্থাপত্যগত মূত্রাণ ব্যবহূত হয়, অন্যথায় হেমোডাইক্রেটিন ইও টক্সিড্রমেস ল্যাবস উল্লেখযোগ্য এতো ওসম একটি শূন্যতা Accridromes ল্যাবগুলি উল্লেখযোগ্য etoh osms aion reples lactat lutax justical barbs stox stox stox district district not refult serum osmolar respective special spective serom osmolar respect yooproppel Adule res যথা: lactat expative espect calstat History of the dose fomepizole mg kgg ex ed ethilin glycol vs methanol ingesation with diazepam mg x etoh revelown e

In [5]:
df['text'][0][:200]

'CHIEF COMPLAINT: AMS, concern for toxic alcohol ingestion\n\nPRESENT ILLNESS: Mr. [**Known lastname 27389**] is a 39 year old man with h/o EtOH and polysubstance abuse, seizure disorder, who was found t'

In [8]:
text_bengali = translate(df['text'][0][:300])
print(text_bengali)

CHIEF COMPLAINT: AMS, AMS, বিষাক্ত মদ দ্রব্যের জন্য উদ্বেগ: Mr. [* চিন্তিত শেষ নাম ২৭৩৮৯*] হিউ/ও ইটাহ এবং পলিসুফস্টেন্সের সহিত ৩৯ বছর বয়সী একজন পুরুষ, যিনি আগে আইসিইউতে তার অংশীদারের সাথে সাক্ষাৎ করার সময় অস্বাভাবিকভাবে আবিষ্কার করেছিলেন।


In [17]:
print(text[:300])

chief complaint ams concern toxic alcohol ingestion present illness mr year old man ho etoh polysubstance abuse seizure disorder found unresponsive visiting partner icu earlier today patient visiting partner icu earlier today awake conversant morning acute complaints noted sleeping floor walked chai


In [18]:
text_bengali = translate(text[:300])
print(text_bengali)

প্রধান অভিযোগ হল, বিষাক্ত মদ ব্যথার জন্য উদ্বেগজনক রোগব্যাধি বিদ্যমান বয়স্ক পুরুষ হোস পলিসস্ট্যান্সের নির্যাতন দুর্ঘটনার রোগে আক্রান্ত অংশীদার আইকুর আগে আজ রাতে জেগে থাকা রোগীর সাথে দেখা করতে যাওয়া সহযোগী আইকুর সাথে দেখা হয়। আজ সকালে ঘুমানো মেঝে চুইয়ের তীব্র অভিযোগ উল্লেখ করা হয়েছে।


In [50]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-m_pq6k6x
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-m_pq6k6x
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 4.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6859 sha256=6d4a471ccfdbaaddf3cc232b20a687c4a63f05bda1b9b761d97c74f8e3d7bef8
  Stored in directory: /tmp/pip-ephem-wheel-cache-rmee1boa/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=cb0073ccc0928c6265498e10e6e4f8290f2

In [51]:
import pandas as pd
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize

In [52]:
# Load the pretrained model and tokenizer
model_name = 'csebuetnlp/banglat5_nmt_en_bn'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

tokenizer_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [53]:
# Function to translate text
def translate(text):
    # Normalize the text
    normalized_text = normalize(text)
    
    # Tokenize and move to device
    inputs = tokenizer(normalized_text, return_tensors='pt', padding=True, truncation=True).to(device)
    
    # Generate translation
    translated = model.generate(**inputs)
    
    # Decode the translated text
    translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    return translated_text[0]

In [73]:
df['text'][0][:300]

'CHIEF COMPLAINT: AMS, concern for toxic alcohol ingestion\n\nPRESENT ILLNESS: Mr. [**Known lastname 27389**] is a 39 year old man with h/o EtOH and polysubstance abuse, seizure disorder, who was found to be unresponsive while visiting his partner in the ICU earlier today. . The patient was visiting hi'

In [74]:
text_bengali = translate(df['text'][0][:300])
print(text_bengali)

শিশুরোগের বিবরণ: এএমএস, বিষাক্ত মদ পান নিয়ে উদ্বেগ বর্তমান সমস্যা: জনাব [*পরিচিত শেষ নাম ২৭৩৮৯*] একজন ৩৯ বছর বয়স্ক ব্যক্তি যার এইচ/ও ইটিওএইচ এবং পলি সাবস্ট্যান্স অপব্যবহার, সিজার্ড ডিসঅর্ডার রয়েছে, যাকে আজ সকালে আইসিইউতে তার সঙ্গীর সাথে দেখা করার সময় সাড়া দিতে দেখা যায়নি। রোগী হাই-স্কুলে যাচ্ছিলেন।


In [64]:
import requests

def translate_text(text, source_lang="en", target_lang="bn"):
    url = "https://libretranslate.com/translate"
    payload = {
        "q": text,
        "source": source_lang,
        "target": target_lang,
        "format": "text"
    }
    headers = {
        "Content-Type": "application/json"
    }
    response = requests.post(url, json=payload, headers=headers)
    return response.json()

text_to_translate = "Hello, how are you?"
translated_text = translate_text(text_to_translate)
print(translated_text)


{'error': 'Visit https://portal.libretranslate.com to get an API key'}


In [66]:
!pip install argostranslate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 16.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 44.1 MB/s eta 0:00:00:00:0100:01
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895240 sha256=5937039d388e7b6940e1c0f71d3caeaa9582ba7a9ac647e16c835cc8d7fae368
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [78]:
import argostranslate.package
import argostranslate.translate

from_code = "en"
to_code = "bn"

# Download and install Argos Translate package
argostranslate.package.update_package_index()
available_packages = argostranslate.package.get_available_packages()
package_to_install = next(
    filter(
        lambda x: x.from_code == from_code and x.to_code == to_code, available_packages
    )
)
argostranslate.package.install_from_path(package_to_install.download())

# Translate
translatedText = argostranslate.translate.translate(text, from_code, to_code)
print(translatedText)
# '¡Hola Mundo!'

যখন Womannatinper-এ Worknominugony de deop যখন Worypory de de denomaponypony de de denonoponing Warmound Wardound Worpuge de de de deponazonazonade de de deponazon to de de denonononononoper-র Wory de uge de deponazonapon'র সাথে যখন Warmav- রাসম্র হতাশ হবেন তখন যখন Wonaper ugnopy deopy de debound Ezazazazoonazazazy deboinazonazonapt de de de ut de dazy de de de de dazonazonazonazonazoicazoickazy.
